In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((3140, 13), (3140, 13))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove the column that specifies which data fold we used
df = df.drop(['fold', 'AL', 'PL', 'vit', 'model', 'data', 'ALL', 'method', 'ratio', 'kw'], axis = 1)

cols = sorted(df.columns.tolist())
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

,dataset,metric,value,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,3.84,5,0.0005,64
1,UCM-CLS,zeroshot-val-top1,7.14,5,0.0005,64
2,RSICD,image_to_text_R@1,0.18,5,0.0005,64
3,RSICD,image_to_text_R@5,0.82,5,0.0005,64
4,RSICD,image_to_text_R@10,1.46,5,0.0005,64
...,...,...,...,...,...,...
3135,Sydney,image_to_text_R@5,5.23,35,0.0005,64
3136,Sydney,image_to_text_R@10,11.27,35,0.0005,64
3137,Sydney,text_to_image_R@1,1.41,35,0.0005,64
3138,Sydney,text_to_image_R@5,7.85,35,0.0005,64


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_grouped)

value                 
                                                       mean        std count
dataset   epochs metric             bs  lr                                  
RSICD     5      image_to_text_R@1  64  0.000005   1.665556   0.388269     9
                                        0.000050   4.936667   0.503835     9
                                        0.000500   0.140000   0.047434     9
                                    128 0.000005   1.208000   0.300533     5
                                        0.000050   3.546000   0.686462     5
                                        0.000500   0.108000   0.040249     5
                 image_to_text_R@10 64  0.000005  14.514444   1.001301     9
                                        0.000050  30.285556   1.129691     9
                                        0.000500   1.065556   0.210423     9
                                    128 0.000005   9.362000   0.430198     5
                                        0.000050  26.216000   1.880247     5
                                        0.000500   0.950000   0.155403     5
                 image_to_text_R@5  64  0.000005   7.708889   0.921038     9
                                        0.000050  17.507778   0.890179     9
                                        0.000500   0.538889   0.154389     9
                                    128 0.000005   5.194000   0.526479     5
                                        0.000050  14.952000   1.029791     5
                                        0.000500   0.532000   0.075299     5
                 text_to_image_R@1  64  0.000005   1.320000   0.289353     9
                                        0.000050   4.171111   0.587057     9
                                        0.000500   0.090000   0.000000     9
                                    128 0.000005   0.930000   0.152151     5
                                        0.000050   3.220000   0.708273     5
                                        0.000500   0.090000   0.000000     5
                 text_to_image_R@10 64  0.000005  12.991111   0.624689     9
                                        0.000050  28.366667   1.148390     9
                                        0.000500   0.925556   0.127780     9
                                    128 0.000005   8.758000   0.404314     5
                                        0.000050  22.796000   0.755599     5
                                        0.000500   0.892000   0.040249     5
                 text_to_image_R@5  64  0.000005   6.844444   0.519112     9
                                        0.000050  16.087778   1.078445     9
                                        0.000500   0.480000   0.039686     9
                                    128 0.000005   4.296000   0.412468     5
                                        0.000050  12.762000   0.754566     5
                                        0.000500   0.460000   0.000000     5
          15     image_to_text_R@1  64  0.000005   3.352222   0.320966     9
                                        0.000050   7.058889   0.648237     9
                                        0.000500   0.211111   0.253547     9
                                    128 0.000005   2.652000   0.509186     5
                                        0.000050   6.436000   0.459162     5
                                        0.000500   0.090000   0.063640     5
                 image_to_text_R@10 64  0.000005  25.136667   1.526499     9
                                        0.000050  34.868889   1.566560     9
                                        0.000500   2.547778   2.000224     9
                                    128 0.000005  20.988000   0.587256     5
                                        0.000050  33.510000   1.868315     5
                                        0.000500   1.132000   0.465908     5
                 image_to_text_R@5  64  0.000005  14.340000   0.709049     9
                                        0.000050  

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_2 = df_grouped.reset_index()
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
df_2 = df_2[(df_2['epochs'] > 0) ] # &(df_2['lr']==5e-5)
df_2 = df_2.sort_values(['metric', 'dataset', ('value', 'mean')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_2)

dataset epochs              metric   bs        lr      value  \
                                                               mean   
41       RSICD     15   image_to_text_R@1  128  0.000500   0.090000   
5        RSICD      5   image_to_text_R@1  128  0.000500   0.108000   
2        RSICD      5   image_to_text_R@1   64  0.000500   0.140000   
77       RSICD     25   image_to_text_R@1  128  0.000500   0.182000   
38       RSICD     15   image_to_text_R@1   64  0.000500   0.211111   
113      RSICD     35   image_to_text_R@1  128  0.000500   0.218000   
74       RSICD     25   image_to_text_R@1   64  0.000500   0.335556   
110      RSICD     35   image_to_text_R@1   64  0.000500   1.171667   
3        RSICD      5   image_to_text_R@1  128  0.000005   1.208000   
0        RSICD      5   image_to_text_R@1   64  0.000005   1.665556   
39       RSICD     15   image_to_text_R@1  128  0.000005   2.652000   
75       RSICD     25   image_to_text_R@1  128  0.000005   3.164000   
36       RSICD     15   image_to_text_R@1   64  0.000005   3.352222   
4        RSICD      5   image_to_text_R@1  128  0.000050   3.546000   
111      RSICD     35   image_to_text_R@1  128  0.000005   4.570000   
72       RSICD     25   image_to_text_R@1   64  0.000005   4.925556   
1        RSICD      5   image_to_text_R@1   64  0.000050   4.936667   
108      RSICD     35   image_to_text_R@1   64  0.000005   5.338000   
40       RSICD     15   image_to_text_R@1  128  0.000050   6.436000   
37       RSICD     15   image_to_text_R@1   64  0.000050   7.058889   
73       RSICD     25   image_to_text_R@1   64  0.000050   7.811111   
76       RSICD     25   image_to_text_R@1  128  0.000050   8.228000   
109      RSICD     35   image_to_text_R@1   64  0.000050   8.610000   
112      RSICD     35   image_to_text_R@1  128  0.000050   8.648000   
209     Sydney     15   image_to_text_R@1  128  0.000500   0.160000   
170     Sydney      5   image_to_text_R@1   64  0.000500   0.200000   
245     Sydney     25   image_to_text_R@1  128  0.000500   0.240000   
173     Sydney      5   image_to_text_R@1  128  0.000500   0.280000   
281     Sydney     35   image_to_text_R@1  128  0.000500   0.482000   
206     Sydney     15   image_to_text_R@1   64  0.000500   0.557778   
242     Sydney     25   image_to_text_R@1   64  0.000500   0.623333   
278     Sydney     35   image_to_text_R@1   64  0.000500   1.473333   
171     Sydney      5   image_to_text_R@1  128  0.000005   1.570000   
168     Sydney      5   image_to_text_R@1   64  0.000005   2.347778   
207     Sydney     15   image_to_text_R@1  128  0.000005   2.374000   
172     Sydney      5   image_to_text_R@1  128  0.000050   2.698000   
204     Sydney     15   image_to_text_R@1   64  0.000005   3.085556   
243     Sydney     25   image_to_text_R@1  128  0.000005   3.098000   
169     Sydney      5   image_to_text_R@1   64  0.000050   3.175556   
240     Sydney     25   image_to_text_R@1   64  0.000005   3.331111   
205     Sydney     15   image_to_text_R@1   64  0.000050   3.376667   
276     Sydney     35   image_to_text_R@1   64  0.000005   3.500000   
279     Sydney     35   image_to_text_R@1  128  0.000005   3.662000   
244     Sydney     25   image_to_text_R@1  128  0.000050   3.864000   
208     Sydney     15   image_to_text_R@1  128  0.000050   3.982000   
277     Sydney     35   image_to_text_R@1   64  0.000050   4.268000   
241     Sydney     25   image_to_text_R@1   64  0.000050   4.360000   
280     Sydney     35   image_to_text_R@1  128  0.000050   4.666000   
314        UCM      5   image_to_text_R@1   64  0.000500   0.480000   
317        UCM      5   image_to_text_R@1  128  0.000500   0.668000   
389        UCM     25   image_to_text_R@1  128  0.000500   0.668000   
353        UCM     15   image_to_text_R@1  128  0.000500   0.670000   
386        UCM     25   image_to_text_R@1   64  0.000500   1.164444   
350        UCM     15   image_to_text_R@1   64  0.000500   1.376667   
425        UCM     35   image_t